In [180]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [122]:
data = pd.read_csv("./합격자 스펙 데이터_은행금융.csv", encoding = "euc-kr")
data

,Unnamed: 0,합격회사,기업정보,지원분야,출신전공,스펙지수,학점,토익,토익스피킹,OPIC,외국어(기타),자격증,해외경험,인턴,수상내역,교내/사회/봉사,종합정보,종목
0,1,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2018년 하반기 \n금융 보험영업,수도권4년제 어문학계열 중어중국학과,515,3.75/4.5,900점,Lv7,-,1개,-,2회,-,-,1회,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
1,2,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2016년 하반기 \n기획 전략 경영,서울4년제 상경계열 경영계열,483,3.54/4.5,850점,-,IH,-,1개,-,-,1회,2회,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
2,3,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2016년 하반기 \n사무보조 문서작성,"지방4년제 예/체능학계열 성악,뮤지컬",222,3.85/4.5,-,-,-,-,2개,-,-,-,-,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
3,4,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2015년 상반기 \n비서 안내,대졸4년제 공학계열 컴퓨터학부,141,3.22/4.5,-,-,-,-,-,-,-,-,-,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
4,5,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2013년 하반기 \n금융 보험영업,지방4년제 법학계열 법경찰,203,3/4.5,-,-,-,-,2개,-,-,-,-,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32775,32776,우리신용정보㈜,"<span>은행·금융</span>,<span>395억 6,765만원</span>,<...",2007년 상반기 \n경리 회계 결산,초대졸 정보통신,83,-,-,-,-,-,1개,-,-,-,-,"우리신용정보㈜<span>은행·금융</span>,<span>395억 6,765만원</...","은행·금융,"
32776,32777,우리신용정보㈜,"<span>은행·금융</span>,<span>395억 6,765만원</span>,<...",2007년 상반기 \n채권 심사 보험 보상,고졸,30,-,-,-,-,-,-,-,-,-,-,"우리신용정보㈜<span>은행·금융</span>,<span>395억 6,765만원</...","은행·금융,"
32777,32778,우리신용정보㈜,"<span>은행·금융</span>,<span>395억 6,765만원</span>,<...",2007년 상반기 \n고객상담 인바운드,수도권4년제 예/체능학계열 연극,162,4.16/4.5,-,-,-,-,-,-,-,-,-,"우리신용정보㈜<span>은행·금융</span>,<span>395억 6,765만원</...","은행·금융,"
32778,32779,우리신용정보㈜,"<span>은행·금융</span>,<span>395억 6,765만원</span>,<...",2007년 상반기 \n채권 심사 보험 보상,지방4년제 상경계열 무역,136,3/4.5,-,-,-,-,-,-,-,-,-,"우리신용정보㈜<span>은행·금융</span>,<span>395억 6,765만원</...","은행·금융,"


In [177]:
data_modi = data.copy()

import re
data_modi = data_modi.drop("Unnamed: 0",axis=1)
companySize = []
for i in range (0,data_modi.shape[0]) : 
    #지원정보 단순화
    data_modi.iloc[i,2] = data_modi.iloc[i,2][11:99]
    #기업정보 단순화 (참고로 매출액 400억원 이하가 중소기업, 80억원 이하가 소기업, 대기업은 매출액 5조 이상?)
    ## 기업 규모 추출
    imsi = re.split("</span>,<span>",data.iloc[i,2])[1].replace(",","")
    if ("조" in imsi) & ("억" in imsi) & ("만원" in imsi) :
        imsi2 = re.split("조|억|만원",imsi)[0:3]
        #숫자로 바꾸기
        total = int(imsi2[0])*(10**12)+int(imsi2[1])*(10**8)+int(imsi2[2])*(10**4)
        companySize.append(total)
    elif ("억" in imsi) & ("만원" in imsi) : 
        imsi2 = re.split("억|만원",imsi)[0:2]
        total = int(imsi2[0])*(10**8)+int(imsi2[1])*(10**4)
        companySize.append(total)
    elif ("억" in imsi) & ("만" in imsi) : 
        imsi2 = re.split("억|만",imsi)[0:2]
        total = int(imsi2[0])*(10**8)+int(imsi2[1])*(10**4)
        companySize.append(total)
    elif ("조" in imsi) & ("억천" in imsi) : #억천을 그냥 억으로 취급함
        imsi2 = re.split("조|억천",imsi)[0:2]
        total = int(imsi2[0])*(10**12)+int(imsi2[1])*(10**8)
        companySize.append(total)
        companySizeisNull.append(0)
    elif ("조" in imsi) & ("억원" in imsi) : #억천을 그냥 억으로 취급함
        imsi2 = re.split("조|억원",imsi)[0:2]
        total = int(imsi2[0])*(10**12)+int(imsi2[1])*(10**8)
        companySize.append(total)
    elif ("조" in imsi) & ("억" in imsi) : #억천을 그냥 억으로 취급함
        imsi2 = re.split("조|억",imsi)[0:2]
        total = int(imsi2[0])*(10**12)+int(imsi2[1])*(10**8)
        companySize.append(total)
    elif ("천억" in imsi) :
        imsi2 = re.split("천억",imsi)[0:2]
        total = int(imsi2[0])*(10**8)*(1000)
        companySize.append(total)
    elif ("억" in imsi) :
        imsi2 = re.split("억",imsi)[0:2]
        total = int(imsi2[0])*(10**8)
        companySize.append(total)
    else : #기업 규모액이 없음
        companySize.append(0)
    ##학력 컨버팅
    if ("4년제" in data_modi.iloc[i,3]):
        data_modi.iloc[i,3] = 2  
    elif ("대학원" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 3
    elif ("해외대학원" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 3
    elif ("해외대학" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 2
    elif ("초대졸" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 1
    elif ("고졸" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 0
    
    ##학점 컨버팅
    if (data_modi.iloc[i,5] != "-") :
        data_modi.iloc[i,5] = float(data_modi.iloc[i,5].replace("/4.5",""))    
    else :
        data_modi.iloc[i,5] = 0
    
    ##토익 컨버팅
    if (data_modi.iloc[i,6] != "-") :
        data_modi.iloc[i,6] = int(data_modi.iloc[i,6].replace("점",""))    
    else :
        data_modi.iloc[i,6] = 0
    ##토익 스피킹 컨버팅
    if (data_modi.iloc[i,7] == "Lv1") :
        data_modi.iloc[i,7] = 1
    elif (data_modi.iloc[i,7] == "Lv2") :
        data_modi.iloc[i,7] = 2
    elif (data_modi.iloc[i,7] == "Lv3") :
        data_modi.iloc[i,7] = 3
    elif (data_modi.iloc[i,7] == "Lv4") :
        data_modi.iloc[i,7] = 4
    elif (data_modi.iloc[i,7] == "Lv5") :
        data_modi.iloc[i,7] = 5
    elif (data_modi.iloc[i,7] == "Lv6") :
        data_modi.iloc[i,7] = 6
    elif (data_modi.iloc[i,7] == "Lv7") :
        data_modi.iloc[i,7] = 7
    elif (data_modi.iloc[i,7] == "Lv8") :
        data_modi.iloc[i,7] = 8
    else :
        data_modi.iloc[i,7] = 0

    ##오픽 컨버팅
    if (data_modi.iloc[i,8] == "NL") :
        data_modi.iloc[i,8] = 1
    elif (data_modi.iloc[i,8] == "NM") :
        data_modi.iloc[i,8] = 2
    elif (data_modi.iloc[i,8] == "NH") :
        data_modi.iloc[i,8] = 3
    elif (data_modi.iloc[i,8] == "IL") :
        data_modi.iloc[i,8] = 4
    elif (data_modi.iloc[i,8] == "IM1") :
        data_modi.iloc[i,8] = 5
    elif (data_modi.iloc[i,8] == "IM2") :
        data_modi.iloc[i,8] = 6
    elif (data_modi.iloc[i,8] == "IM3") :
        data_modi.iloc[i,8] = 7
    elif (data_modi.iloc[i,8] == "IH") :
        data_modi.iloc[i,8] = 8
    elif (data_modi.iloc[i,8] == "AL") :
        data_modi.iloc[i,8] = 9
    else :
        data_modi.iloc[i,8] = 0
        
    ##제2외국어 컨버팅
    if (data_modi.iloc[i,9] != "-") :
        data_modi.iloc[i,9] = int(data_modi.iloc[i,9].replace("개",""))    
    else :
        data_modi.iloc[i,9] = 0
        
    ##자격증 컨버팅
    if (data_modi.iloc[i,10] != "-") :
        data_modi.iloc[i,10] = int(data_modi.iloc[i,10].replace("개",""))    
    else :
        data_modi.iloc[i,10] = 0

    ##해외경험 컨버팅
    if (data_modi.iloc[i,11] != "-") :
        data_modi.iloc[i,11] = int(data_modi.iloc[i,11].replace("회",""))    
    else :
        data_modi.iloc[i,11] = 0
        
    ##인턴 컨버팅
    if (data_modi.iloc[i,12] != "-") :
        data_modi.iloc[i,12] = int(data_modi.iloc[i,12].replace("회",""))    
    else :
        data_modi.iloc[i,12] = 0

    ##수상내역 컨버팅
    if (data_modi.iloc[i,13] != "-") :
        data_modi.iloc[i,13] = int(data_modi.iloc[i,13].replace("회",""))    
    else :
        data_modi.iloc[i,13] = 0

    ##사회봉사 컨버팅
    if (data_modi.iloc[i,14] != "-") :
        data_modi.iloc[i,14] = int(data_modi.iloc[i,14].replace("회",""))    
    else :
        data_modi.iloc[i,14] = 0
    
        
data_modi.insert(2,"기업 총액(원)",companySize)
data_modi = data_modi.drop("기업정보",axis=1)
data_modi = data_modi.drop("종합정보",axis=1)
data_modi = data_modi.drop("종목",axis=1)


data_modi


,합격회사,기업 총액(원),지원분야,출신전공,스펙지수,학점,토익,토익스피킹,OPIC,외국어(기타),자격증,해외경험,인턴,수상내역,교내/사회/봉사
0,(주)하나저축은행,95918580000,금융 보험영업,2,515,3.75,900,7,0,1,0,2,0,0,1
1,(주)하나저축은행,95918580000,기획 전략 경영,2,483,3.54,850,0,8,0,1,0,0,1,2
2,(주)하나저축은행,95918580000,사무보조 문서작성,2,222,3.85,0,0,0,0,2,0,0,0,0
3,(주)하나저축은행,95918580000,비서 안내,2,141,3.22,0,0,0,0,0,0,0,0,0
4,(주)하나저축은행,95918580000,금융 보험영업,2,203,3,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32775,우리신용정보㈜,39567650000,경리 회계 결산,1,83,0,0,0,0,0,1,0,0,0,0
32776,우리신용정보㈜,39567650000,채권 심사 보험 보상,0,30,0,0,0,0,0,0,0,0,0,0
32777,우리신용정보㈜,39567650000,고객상담 인바운드,2,162,4.16,0,0,0,0,0,0,0,0,0
32778,우리신용정보㈜,39567650000,채권 심사 보험 보상,2,136,3,0,0,0,0,0,0,0,0,0


In [205]:
X = data_modi.iloc[:,[3,5,6,7,8,9,10,11,12,13,14]].copy()
y = data_modi.iloc[:,0].copy()
y2 = pd.get_dummies(y)
X
y

0        (주)하나저축은행
1        (주)하나저축은행
2        (주)하나저축은행
3        (주)하나저축은행
4        (주)하나저축은행
           ...    
32775      우리신용정보㈜
32776      우리신용정보㈜
32777      우리신용정보㈜
32778      우리신용정보㈜
32779      우리신용정보㈜
Name: 합격회사, Length: 32780, dtype: object

In [211]:
#라벨 인코딩

y_label = y.copy()
indexdata = range(0,len(y_label.unique()))
imsiSeries = pd.Series(data=indexdata, index=y_label.unique())
y_label = y_label.map(imsiSeries)
indexdata
y_label

0          0
1          0
2          0
3          0
4          0
        ... 
32775    118
32776    118
32777    118
32778    118
32779    118
Name: 합격회사, Length: 32780, dtype: int64

In [212]:
#트레인셋 나누기

X_train, X_test, y_train, y_test = train_test_split(X,y_label, random_state = 2, test_size=0.3)

In [213]:
knn = KNeighborsClassifier(n_neighbors = 3)

In [214]:
knn.fit(X_train,y_train)

y_predict = knn.predict(X_test)

In [215]:
knn.score(X_train,y_train)

0.19550248409308812

4778    -14
14231   -31
11392    -8
27459   -59
8978    -21
         ..
9620    -21
19245   -38
28538   -59
22560   -33
16803     8
Name: 합격회사, Length: 9834, dtype: int64

,합격회사,기업 총액(원),지원분야,출신전공,스펙지수,학점,토익,토익스피킹,OPIC,외국어(기타),자격증,해외경험,인턴,수상내역,교내/사회/봉사
0,(주)하나저축은행,95918580000,금융 보험영업,2,515,3.75,900,7,0,1,0,2,0,0,1
1,(주)하나저축은행,95918580000,기획 전략 경영,2,483,3.54,850,0,8,0,1,0,0,1,2
2,(주)하나저축은행,95918580000,사무보조 문서작성,2,222,3.85,0,0,0,0,2,0,0,0,0
3,(주)하나저축은행,95918580000,비서 안내,2,141,3.22,0,0,0,0,0,0,0,0,0
4,(주)하나저축은행,95918580000,금융 보험영업,2,203,3,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32775,우리신용정보㈜,39567650000,경리 회계 결산,1,83,0,0,0,0,0,1,0,0,0,0
32776,우리신용정보㈜,39567650000,채권 심사 보험 보상,0,30,0,0,0,0,0,0,0,0,0,0
32777,우리신용정보㈜,39567650000,고객상담 인바운드,2,162,4.16,0,0,0,0,0,0,0,0,0
32778,우리신용정보㈜,39567650000,채권 심사 보험 보상,2,136,3,0,0,0,0,0,0,0,0,0


array(['금융 보험영업', '기획 전략 경영', '사무보조 문서작성', '비서 안내', '사무 총무 법무',
       '리서치 통계 설문', '아웃바운드TM', '배송 택배 운송', '채권 심사 보험 보상', '응용프로그래머',
       '고객상담 인바운드', '경리 회계 결산', '인사 노무 교육', '영업관리 지원 영업기획', '재무 세무 IR',
       'ERP 시스템분석 설계', '회계 세무 CPA', '시설 보안 경비 안전', '제품 서비스영업',
       '마케팅 광고 분석', '외국어 번역 통역', '', '법인영업', '컨텐츠 사이트운영', '학원상담 관리 운영',
       '홍보 PR', '유통 물류 재고', '네트워크 서버 보안', '단순홍보 회원관리', 'QA 테스터 검증',
       '소프트웨어 하드웨어', '웹기획 PM', '해외영업 무역영업', '출판 편집디자인', '통신기술 네트워크구축',
       '경영분석 컨설턴트', '설치 정비 A/S', 'HTML 퍼블리싱 UI개발', '법률 특허 상표',
       'IT 솔루션 기술영업', '부동산 중개 분양 경매', '웹프로그래머', 'IT 디자인 컴퓨터강사',
       '영상 사진 촬영', '판매 서빙 매장관리', '웹디자인', '시스템프로그래머', '감독 연출 PD',
       '작가 시나리오', '그래픽디자인 CG', '광고 시각디자인', '이벤트 웨딩 도우미', '상품기획 MD',
       '아나운서 리포터 성우', '요리 영양 제과제빵 바리스타', '빅데이터 AI(인공지능)', '자동차 조선 기계',
       '반도체 디스플레이', '초중고 특수학교', '기자', '사회복지 요양보호 자원봉사', '의사 치과 한의사',
       '학습지 과외 방문교사', '건축 설계 인테리어', '자격증 기술 전문교육', '뷰티 미용 애완', '음악 음향',
       '구매 자재', '노무 헤드헌터 직업상담', '시공 현장 감리 공무', '생

'2016년 하반기 \n기획 전략 경영'

'은행·금융</span>,959억 1,858만원</span>,154명 재직중</span>'

'<span>은행·금융</span>,<span>959억 1,858만원</span>,<span>154명 재직중</span>'

'959억 1858만원154명 재직중'

1조 430억 6827만원


['1', ' 430', ' 6827']

TypeError: can only concatenate str (not "int") to str